# XGBoost (Extreme Gradient Boosting)

Segundo os autores da ferramenta, XGBboost (**Extreme Gradient Boosting**) é um sistema de _tree bosting_ escalável de ponta a ponta, ou, em suas palavras:

> "a scalable end-to-end tree boosting system called XGBoost"

Certo é um sistema baseada em _tree boosting_. Porém, qual o significado do termo "**boosting**"?


## Boosting

Segundo **Zhi-Hua Zhou p. 23**, livro *Ensemble Methods: Foundations and Algorithms*:
> o termo _Boosting_ se refere a uma família de algoritmos capazes de converter _weak learners_ em _strong learners_

Certo, mas o que seriam _weak learners_ e *strong learners*?  
De maneira simplificada \[e intuitiva\]:

**1. Definição: um _weak learner_ é um modelo com desempenho ligeiramente superior ao método aleatório (advinhação aleatória)**

Certo, e quanto a um *strong learner*?
Similarmente:

**2. Definição: um _strong learner_ é um modelo muito próximo à perfeição**

A partir desses dois conceitos, surge a ideia de _converter_ **weak learner** &rarr; **strong learner**. A partir desse conceito, provou-se \[por contrução\] a equivalência entre essas classes. E a tal contrução foi atribuído o nome _**boosting**_.  

A título de exemplo, temos a técnica *gradient boosting* a qual consiste em, dado um *comitê* composto por um modelo, suas predições referentes ao um conjunto de dados e os rótulos corretos, adicionar ao comitê um novo modelo o qual *corrija* os erros do classificador passado. Esse processo de inclusão de novos modelos é repetido até que algum critério de parada seja atendido.

<!-- [intro-boosting]: https://cseweb.ucsd.edu/~yfreund/papers/IntroToBoosting.pdf -->

## Gradient Boosting

> Obs: para os fins dessa explicação, considere tratar-se de um problema de classificação. A definição pode ser trivialmente estendida para problemas de regressão.

A ideia do referido **gradient boosting** é a construção de um **_strong learner_** a partir da _adição_ de novos classificadores a partir de um classificador base. Cada novo classificador deve reduzir os _erros cometidos pelo classificador anterior_ (grosso modo, "compensá-los"). Desse modo, cada novo classificador adicionado tem por objetivo corrigir as predições do antigo classificador. Esse procedimento continua até que se atinja um critério de parada.

> Nota: é possível a ocorrência de _overfitting_ caso o procedimento ocorra demasiadas vezes.

Ou seja: **_gradient boosting_** tem por objetivo a contrução de uma sequência de _comitês_ de classificadores (usualmente, árvores de decisão) cujo objetivo é ser cada vez melhor que o comitê anterior.  

Há pesquisas acerca da utilização de [redes neurais "rasas" enquanto _weak learners_][nn-weak-learner].

[nn-weak-learner]: https://arxiv.org/pdf/2002.07971.pdf

# Extreme Gradient Boosting (XGBoost)

Resumidamente, **XGBoost** é um framework que implementa **gradient boosting**, porém com um foco em _performance e escalabilidade_.  

Por exemplo, o sistema é _"sparsity awareness"_ (ou seja, a esparsidade dos dados é levada em consideração pelo sistema a fim de melhor gerenciar os recursos computacionais disponíveis) garantindo assim uma maior escalabilidade. 
Além disso, o modo de funcionamento da cache (pg.2 artigo XGBoost) é levado em consideração para construção de estruturas de dados possibilitando ganho de performance em até 2x-3x.

## Visão geral

XGBoost é não apenas um algoritmo, mas um *sistema completo* baseada no algoritmo **gradient boosting**. O funcionamento deste consiste em, dado um modelo inicial _fraco_ (weak learner), ir adicionando novos modelos ao comitê esperando pela correção dos erros passados através dessas novas adições. A cada iteração do método, um novo modelo (nesse caso, uma *árvore de regressão*) é adicionada ao comitê e a previsão dele é dada por

\begin{equation*}
\widehat{y_1}=\phi(x_i)=\sum_{k=1}^Kf_k(x_i),f_k\in \boldsymbol{\mathcal{F}}
\label{eq1} \tag{1}
\end{equation*}


Uma vez especificado como é realizada a predição, é necessário definir alguma função objetivo para minimização. Tal função é

\begin{equation*}
\boldsymbol{\mathfrak{L}}^{(t)}=\sum_{i=1}^nl(y_i,\widehat{y}_i^{(t-1)}+f_t(x_i))+\Omega(f_t))
\label{eq2} \tag{2}
\end{equation*}


`l` é uma **função de perda** *convexa* (para que a aplicação do gradienta faça sentido), `y` é o valor real, `ŷ` é a previsão do comitê e  `Ômega` é um fator de *regularização* que `penaliza a última árvore adicionada ao comitê  por sua complexidade`.  

A estratégia é escolher uma função
\begin{equation*}
f_i(x_i)
\label{eq3} \tag{3}
\end{equation*}
que minimize L.

Após uma série de operações matemáticas incluindo séries de Taylor para aproximação de `l` e computação de pesos ótimo para cada árvore `f`, obtem-se um espécie de *medida de impureza* para as árvores de decisão (obs: `g` e `h` são os gradientes de 1ª e 2ª ordem da função de perda `l`). Essa medida,

\begin{equation*}
\boldsymbol{\mathfrak{L}}_{split}=\frac{1}{2} \left [\frac{(\sum_{i\in I_L }g_i)^2)}{\sum_{i\in I_L}h_i+\lambda} +\frac{(\sum_{i\in I_R}g_i)^2}{\sum_{i\in I_R}h_i+\lambda} - \frac{(\sum_{i\in I}g_i)^2}{\sum_{i\in I}h_i+\lambda} \right ]-\gamma
\label{eq4} \tag{4}
\end{equation*}

costuma ser utilizada para decidir entre os candidatos para divisão da árvore (é usada no algoritmo de divisão aproximado e exato).  



## Divisão da árvore

Sendo impossível a computação de todas as estruturas de árvores possíveis em tempo razoável, **XGBoost** adota um algoritmo *guloso* para gerar cada árvore. Conforme descrito em seu artigo, existe um "algoritmo guloso exato" para encontrar a estrutura ótima; porém, conforme citado previamente, é computacionalmente inviável fazê-lo caso os dados não caibam em memória principal. Nesses casos, é utilizado um algoritmo *guloso aproximado*.

### Algoritmo exato
Para cada novo nó da árvore, computa todos os possíveis ganhos para todas as divisões possíveis antes de realizar uma divisão, sempre realizando a divisão que trará o maior *score* (como um algoritmo de árvore de decisão genérico). A peculiaridade aqui fica por conta da utilização da fórmula em \eqref{eq4} utilizada durante essa avaliação.


### Algoritmo aproximado
Resumidamente, conforme seção 3.2 do artigo original, o algoritmo funciona com as seguintes etapas:

1. baseado em percentis das *features*, propõe valores a serem usados para divisão da árvores;
2. discretização (contínuo &rarr; intervalos); fazer o mesmo procedimento descrito no item anterior
3. realizar os passos do `algoritmo exato`, com a diferença de que o espaço de busca foi reduzido


### "Sparsity aware"
A forma como é tratada a esparsidade dos dados (valores ausentes, variáveis com muitos zeros, etc) deve fazer parte do algoritmo. Isso é feito por meio de *decisões padrão* em cada nó da árvore. Cada "decisão-padrão" é aprendida a partir dos dados.

<img src="img/default-decision.png">


### Outras otimizações

Além do algoritmo descrito, diversas otimizações são implementadas.  
As mais importantes são:

#### Pré-ordenamento das colunas em uma estrutura referida no artigo como `bloco`
Cada bloco armazenado no formato **CSC** (`compressed columns`, uma forma compacta de armazenar dados utilizada também na biblioteca `scipy`).
A principal contribuição dessa abordagem é não ter que ordenar os dados inúmeras vezes ao longo do processo de construção das árvores.
Além disso, o fato dos dados serem divididos em blocos permite por exemplo sua distribuição entre máquinas, permitindo assim paralelização do acessos aos dados (organizados por blocos em colunas).
#### Acesso ciente da cache
Permite ganho de 2-3x de performance para grandes conjuntos de dados.

#### computação "out-of-core"
Para conjuntos de dados grandes o bastante que não cabem na memória principal, XGBoost utiliza a estratégia de *salvar em disco* os `blocos` de dados em conjunto com a utilização de 1 *thread* independente para carregar esses blocos em memória conforme se faça necessário.


# Referências
0. [Artigo do xgboost][xgboost]
1. [Ensemble methods; Zhi-Hua][ensemble-methods-bib]
2. [Compressed Sparse Column Format (CSC)][csc]
3. [CSC - wikipedia][csc-wiki]
4. [Função de perda][loss-fun]
5. [Função convexa][conv-fun]
6. [Árvores de decisão][dec-tree]
7. [Séries de Taylor][taylor]
8. [Guloso][greedy]
9. [*gradient boosting*][grad-boost]
10. [Ensemble methods][ens-meth]
11. [Ensemble learning][ens-learn]

[xgboost]: https://arxiv.org/pdf/1603.02754.pdf
[ensemble-methods-bib]: https://dl.acm.org/doi/book/10.5555/2381019
[csc]: https://scipy-lectures.org/advanced/scipy_sparse/csc_matrix.html
[csc-wiki]: https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_column_(CSC_or_CCS)
[conv-fun]: https://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_convexa
[loss-fun]: https://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_perda
[dec-tree]: https://medium.com/@gabriel.stankevix/arvore-de-decis%C3%A3o-em-r-85a449b296b2
[taylor]: https://pt.wikipedia.org/wiki/S%C3%A9rie_de_Taylor
[greedy]: https://www.ime.usp.br/~pf/analise_de_algoritmos/aulas/guloso.html
[grad-boost]: https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/
[ens-meth]: https://www.amazon.com/Ensemble-Methods-Foundations-Algorithms-Recognition/dp/1439830037
[ens-learn]: http://www.scholarpedia.org/article/Ensemble_learning